In [ ]:
import torch
import random

'''
The MLP implemented in this notebook is based on the design from
the paper 'A Neural Probabilistic Language Model' (Bengio et al. 2003)
https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

Kaiming initialisation is implemented based on 'Delving Deep into Rectifiers'
(Kaiming et al. 2015) https://arxiv.org/pdf/1502.01852.pdf

Batch normalisation is implemented is based on work from the paper 'Batch
Normalization:  Accelerating Deep Network Training by Reducing Internal 
Covariate Shift' (Ioffe et al. 2015) https://arxiv.org/pdf/1502.03167.pdf
'''

In [ ]:
words = open('data/names.txt', 'r').read().splitlines()

chars = ['.'] + sorted(list(set(''.join(words))))
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for i, s in enumerate(chars)}
vocab_size = len(chars)

In [ ]:
class Linear:

    def __init__(self, nin: int, nout: int, bias: bool=True):
        kaiming_init = 1 / nin**0.5
        self.weights = torch.randn((nin, nout)) * kaiming_init
        self.bias = torch.zeros(nout) if bias else None

    def __call__(self, x: list[float]):
        self.out = x @ self.weights
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weights] + ([] if self.bias is None else [self.bias])


class BatchNorm1D:

    def __init__(self, dims: int, epsilon: float=1e-5, momentum: float=0.1):
        self.epsilon = epsilon
        self.momentum = momentum
        self.training = True
        # Parameters.
        self.gamma = torch.ones(dims)
        self.beta = torch.zeros(dims)
        self.mean_live = torch.zeros(dims)
        self.varience_live = torch.ones(dims)

    def __call__(self, x: list[float]):
        # Forward pass.
        if self.training:
            x_mean = x.mean(0, keepdim=True) # Batch mean.
            x_varience = x.var(0, keepdim=True) # Batch varience.
        else:
            x_mean = self.mean_live
            x_varience = self.varience_live
        x_norm = (x - x_mean) / torch.sqrt(x_varience + self.epsilon) # Normalise to unit varience.
        self.out = self.gamma * x_norm + self.beta
        # Update parameters.
        if self.training:
            with torch.no_grad():
                self.mean_live = (1 - self.momentum) * self.mean_live + self.momentum * x_mean
                self.varience_live = (1 - self.momentum) * self.varience_live + self.momentum * x_varience
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

class Tanh:

    def __call__(self, x: list[float]):
        self.out = torch.tanh(x)
        return self.out

    def parameters(self):
        return []

In [ ]:
# Hyperparameters
block_size = 3 # Context length (characters).
embedding_dims = 10 # Number of dimensions for the embedding space.
batch_size = 32 # Number of examples to process at a time in training.
hidden_layer_size = 100 # Number of neurons in the hidden layer.
init_lr = 0.1 # Initial learning rate.
final_lr = 0.01 # Final learning rate.
max_steps = 200000

def build_dataset(words: list[str]) -> (torch.Tensor, torch.Tensor):

    X, Y = [], []

    for word in words:
        context = [0] * block_size # Padding the context with initial '.' characters.
        for char in word + '.':
            idx = stoi[char]
            X.append(context)
            Y.append(idx)
            context = context[1:] + [idx] # Update context.

    return torch.tensor(X), torch.tensor(Y)

random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

x_train, y_train = build_dataset(words[:n1])
x_val, y_val = build_dataset(words[n1:n2])
x_test, y_test = build_dataset(words[n2:])

In [ ]:
# Embedding matrix.
C = torch.randn((vocab_size, embedding_dims))

# MLP.
layers = [
    Linear(embedding_dims * block_size, hidden_layer_size), BatchNorm1D(hidden_layer_size), Tanh(),
    Linear(hidden_layer_size, hidden_layer_size), BatchNorm1D(hidden_layer_size), Tanh(),
    Linear(hidden_layer_size, hidden_layer_size), BatchNorm1D(hidden_layer_size), Tanh(),
    Linear(hidden_layer_size, hidden_layer_size), BatchNorm1D(hidden_layer_size), Tanh(),
    Linear(hidden_layer_size, hidden_layer_size), BatchNorm1D(hidden_layer_size), Tanh(),
    Linear(hidden_layer_size, vocab_size), BatchNorm1D(vocab_size)
]

# Initialisations.
with torch.no_grad():
    layers[-1].gamma *= 0.1 # Make last layer less confident.
    for layer in layers[:-1]: # For all other layers, apply gain.
        if isinstance(layer, Linear):
            layer.weights *= 5/3

params = [C] + [param for layer in layers for param in layer.parameters()]
for param in params:
    param.requires_grad = True
    
print(f'Number of parameters: {sum(param.nelement() for param in params)}')

In [ ]:
# Gradient descent.
for i in range(max_steps):

    # Constructing batches.
    idx = torch.randint(0, x_train.shape[0], (batch_size, ))

    # Forward pass.
    embedding = C[x_train[idx]] # Embed characters into vectors.
    x = embedding.view(embedding.size(0), -1)
    for layer in layers:
        x = layer(x)
    loss = torch.nn.functional.cross_entropy(x, y_train[idx])

    # Backward pass.
    for layer in layers:
        layer.out.retain_grad()
    for param in params:
        param.grad = None
    loss.backward()

    # Stochastic gradient descent.
    lr = init_lr if i < (max_steps / 2) else final_lr
    for param in params:
        param.data -= lr * param.grad

In [ ]:
def split_loss(split: str) -> None:
    ''' Evaluates the model on the chosen split. '''

    x, y = {
        'train': (x_train, y_train),
        'val': (x_val, y_val),
        'test': (x_test, y_test)
    }[split]
    
    # Forward pass.
    embedding = C[x]
    x = embedding.view(embedding.size(0), -1)
    for layer in layers:
        x = layer(x)
    loss = torch.nn.functional.cross_entropy(x, y)
    print(f'{split.capitalize()} Loss: {loss.data}')

In [ ]:
# Set layers to evaluation mode.
for layer in layers:
  layer.training = False

split_loss('test')
split_loss('val')

In [ ]:
# Sample from the model.
for _ in range(5):
    out = []
    context = [0] * block_size # Initialise context to '...'

    while True:
        # Forward pass.
        embedding = C[torch.tensor([context])]
        x = embedding.view(embedding.size(0), -1)
        for layer in layers:
            x = layer(x)
        logits = x
        probs = torch.nn.functional.softmax(logits, dim=1)

        # Sample from the distribution.
        idx = torch.multinomial(probs, num_samples=1, replacement=True).item()
        context = context[1:] + [idx] # Shift the context window.
        out.append(itos[idx])
        if idx == 0: # If we sample '.', stop.
            break
    print(''.join(out))